In [1]:
import pandas as pd
import json
import numpy as np
from time import sleep
import time
from tqdm import tqdm
import requests
from pydantic import BaseModel
url = open("local_models_path.txt", "r").read()

In [2]:
# Load the test datasets

test_en = pd.read_json("../../datasets/ParlaSent-EN-test/ParlaSent_EN_test.jsonl", lines=True)
test_bcs = pd.read_json("../../datasets/ParlaSent-BSC-test/ParlaSent_BCS_test.jsonl", lines=True)

print(test_en.shape, test_bcs.shape)

(2600, 14) (2600, 14)


In [3]:
def run_local_model(model, prompt, url=url):

	class ReponseStructure(BaseModel):
		sentiment: int

	data = {
	    "model": model,
	    "prompt": prompt,
	    "stream": False,
	    "temperature": 0,
	    "format": ReponseStructure.model_json_schema()
	}

	headers = {"Content-Type": "application/json",}
	response = requests.post(url, json=data, headers=headers)

	return response.json()["response"]

In [4]:
models = ["gemma3:27b", "deepseek-r1:14b", "llama3.3:latest"]

In [5]:
def predict_gpt(df_test_name, gpt_model):

	dfs = {
		"ParlaSent-EN-test": test_en,
		"ParlaSent-BCS-test": test_bcs
	}

	df = dfs[df_test_name]

	responses = []
	
	texts = df["text"].to_list()
	langs = df["lang"].to_list()

	labels_dict = {0: "Negative", 1: "Neutral", 2: "Positive"}

	sentiment_description = {
		"Negative - text that is entirely or predominantly negative":  0, 
		"Neutral - text that only contains non-sentiment-related statements": 1,
		"Positive - text that is entirely or predominantly positive": 2
	}

	start_time = time.time()

	for i in list(zip(texts, langs)):
		text = i[0]
		lang = i[1]

		current_prompt = f"""
			### Task
				Your task is to classify the provided parliamentary text into a sentiment label, meaning that you need to recognize whether the speaker's sentiment towards the topic is negative, neutral, positive or somewhere in between. You will be provided with an excerpt from a parliamentary speech in {lang} language, delimited by single quotation marks. Always provide a label, even if you are not sure.


			### Output format
				Return a valid JSON dictionary with the following key: 'sentiment' and a value should be an integer which represents one of the labels according to the following dictionary: {sentiment_description}.

				Text: '{text}'
		"""

		initial_response= run_local_model(gpt_model, current_prompt, url=url)

		response = initial_response.replace("\n", "")
		response = response.replace("\t", "")

		# Convert the string into a dictionary
		response = json.loads(response)

		# Get out a label
		try:
			predicted = labels_dict[response["sentiment"]]
			responses.append(predicted)
		# add a possibility of something going wrong
		except:
			predicted = initial_response
			print("error with extracting a label:")
			print(initial_response)
			responses.append("Mix")

	end_time = time.time()
	elapsed_time_min = end_time-start_time

	print(f"Prediction finished. It took {elapsed_time_min/60} min for {df.shape[0]} instances - {elapsed_time_min/df.shape[0]} s per instance.")

	# Create a json with results

	current_results = {
		"system": gpt_model,
		"predictions": [
			{
			"train": "NA (zero-shot)",
			"test": "{}".format(df_test_name),
			"predictions": responses,
			}
		]
		}

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format(gpt_model, df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format(gpt_model, df_test_name))

In [6]:
for test in ["ParlaSent-EN-test", "ParlaSent-BCS-test"]:
	for model in models:
		print(model)
		predict_gpt(test, model)


gemma3:27b
Prediction finished. It took 12.55815504391988 min for 2600 instances - 0.28980357793661266 s per instance.
Classification with gemma3:27b on ParlaSent-EN-test finished.
deepseek-r1:14b
Prediction finished. It took 17.92105767329534 min for 2600 instances - 0.4135628693837386 s per instance.
Classification with deepseek-r1:14b on ParlaSent-EN-test finished.
llama3.3:latest
Prediction finished. It took 21.460712520281472 min for 2600 instances - 0.4952472120064956 s per instance.
Classification with llama3.3:latest on ParlaSent-EN-test finished.
gemma3:27b
Prediction finished. It took 13.968113350868226 min for 2600 instances - 0.32234107732772826 s per instance.
Classification with gemma3:27b on ParlaSent-BCS-test finished.
deepseek-r1:14b
Prediction finished. It took 17.320038131872813 min for 2600 instances - 0.3996931876586034 s per instance.
Classification with deepseek-r1:14b on ParlaSent-BCS-test finished.
llama3.3:latest
Prediction finished. It took 28.91738468805949 